In [2]:
import json
import pandas as pd
from IPython.display import Javascript
from IPython.core.display import HTML

data = json.load(open('data/data.json'))
chartData = data['graphData'][0]['chartSeries']
graphData = [chartData[i]['data'] for i in range(len(chartData))]

df = pd.DataFrame(graphData[0]+graphData[1]+graphData[2])
df.columns = ['Timestamp', 'PM 2.5']
df = df.dropna(axis=0, how='any')
df.count()

Timestamp    2685
PM 2.5       2685
dtype: int64

In [4]:
df = df.groupby('Timestamp').agg({'PM 2.5':'min'})['PM 2.5'].reset_index()
d3_data = df.to_json(orient='records')
df.sort_values(['PM 2.5'] ,ascending=False).head()

,Timestamp,PM 2.5
967,1511913600000,169.32
981,1513209600000,152.20
992,1514160000000,148.47
987,1513728000000,108.33
497,1471305600000,104.31


In [5]:
Javascript("window.d3_data={};".format(d3_data))

<IPython.core.display.Javascript object>

In [6]:
%%javascript
require.config({
    paths: {
        d31: '//cdnjs.cloudflare.com/ajax/libs/d3/4.12.0/d3'
    }
});


<IPython.core.display.Javascript object>

In [20]:
# Add the HTML Code here
HTML("""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>AIQ PM 2.5</title>
    
    <style>
        body {
          font: 10px sans-serif;
          shape-rendering: crispEdges;
        }

        .day {
          fill: #fff;
          stroke: #ccc;
        }

        .month {
          fill: none;
          stroke: #000;
          stroke-width: 2px;
        }

        .RdYlGn .q0-11{fill:rgb(165,0,38)}
        .RdYlGn .q1-11{fill:rgb(215,48,39)}
        .RdYlGn .q2-11{fill:rgb(244,109,67)}
        .RdYlGn .q3-11{fill:rgb(253,174,97)}
        .RdYlGn .q4-11{fill:rgb(254,224,139)}
        .RdYlGn .q5-11{fill:rgb(255,255,191)}
        .RdYlGn .q6-11{fill:rgb(217,239,139)}
        .RdYlGn .q7-11{fill:rgb(166,217,106)}
        .RdYlGn .q8-11{fill:rgb(102,189,99)}
        .RdYlGn .q9-11{fill:rgb(26,152,80)}
        .RdYlGn .q10-11{fill:rgb(0,104,55)}
    </style>
</head>
<body>
</body>
</html>
""")

In [19]:
%%javascript
require(['d31'], function(d3) {
    
   var width = 960,
    height = 136,
    cellSize = 17; // cell size

var percent = d3.format(".1%"),
    format = d3.timeFormat("%Y-%m-%d");

var color = d3.scaleQuantize()
    .domain([-.05, .05])
    .range(d3.range(11).map(function(d) { return "q" + d + "-11"; }));

var svg = d3.select("body").selectAll("svg")
    .data(d3.range(1990, 2011))
  .enter().append("svg")
    .attr("width", width)
    .attr("height", height)
    .attr("class", "RdYlGn")
  .append("g")
    .attr("transform", "translate(" + ((width - cellSize * 53) / 2) + "," + (height - cellSize * 7 - 1) + ")");

svg.append("text")
    .attr("transform", "translate(-6," + cellSize * 3.5 + ")rotate(-90)")
    .style("text-anchor", "middle")
    .text(function(d) { return d; });

var rect = svg.selectAll(".day")
    .data(function(d) { return d3.timeDays(new Date(d, 0, 1), new Date(d + 1, 0, 1)); })
  .enter().append("rect")
    .attr("class", "day")
    .attr("width", cellSize)
    .attr("height", cellSize)
    .attr("x", function(d) { return d3.timeWeek.count(d3.timeYear(d), d) * cellSize; })
    .attr("y", function(d) { return d.getDay() * cellSize; })
    .datum(format);

rect.append("title")
    .text(function(d) { return d; });

svg.selectAll(".month")
    .data(function(d) { return d3.timeMonths(new Date(d, 0, 1), new Date(d + 1, 0, 1)); })
  .enter().append("path")
    .attr("class", "month")
    .attr("d", monthPath);

d3.csv("dji.csv", function(error, csv) {
  if (error) throw error;

  var data = d3.nest()
    .key(function(d) { return d.Date; })
    .rollup(function(d) { return (d[0].Close - d[0].Open) / d[0].Open; })
    .map(csv);
``
  console.log('data', data);

  rect.filter(function(d) { return data.has(d); })
      .attr("class", function(d) { return "day " + color(data.get(d)); })
    .select("title")
      .text(function(d) { return d + ": " + percent(data.get(d)); });
});

function monthPath(t0) {
  var t1 = new Date(t0.getFullYear(), t0.getMonth() + 1, 0),
      d0 = t0.getDay(), w0 = d3.timeWeek.count(d3.timeYear(t0), t0),
      d1 = t1.getDay(), w1 = d3.timeWeek.count(d3.timeYear(t1), t1);
  return "M" + (w0 + 1) * cellSize + "," + d0 * cellSize
      + "H" + w0 * cellSize + "V" + 7 * cellSize
      + "H" + w1 * cellSize + "V" + (d1 + 1) * cellSize
      + "H" + (w1 + 1) * cellSize + "V" + 0
      + "H" + (w0 + 1) * cellSize + "Z";
}
    
});


<IPython.core.display.Javascript object>